In [1]:
import random
import networkx as nx
from matplotlib import pyplot as plt
from collections import Counter
import math
from utils.plotTools import plot_qwak
import os
import ast
import numpy as np
import json
import pandas as pd
from sklearn.linear_model import LinearRegression
import cupy as cp


from scripts import load_list_from_file, write_list_to_file, load_or_generate_data, draw_graph, draw_graph_from_adjacency_matrix,print_matrix
from scripts_tempHelix import generate_static_temporal_helix, generate_temporal_helix, multiple_exponential_temporal_helix,direct_sum_triangular_graph, direct_sum_reverse_triangular_graph
from scripts_theoreticalHittingTime import create_transition_matrix
from scripts_tempHelix_CuPy import generate_temporal_helix_cupy

In [2]:
def print_matrix(matrix):
    # Print rows with left and right border
    for row in matrix:
        print('|', end='')
        print(' '.join(format(item, ".2f") for item in row), end=' |\n')
        
figsize = (15, 6)
k = 0.2
draw_self_loops = False

config = {
    'figsize': figsize,
    'k': k,
    'draw_self_loops': draw_self_loops,
    'node_color': 'orange',
    'node_size': 800,
    'normal_edge_color': 'blue',
    'normal_edge_width': 1.5,
    'edge_style': 'solid',
    'self_loop_color': 'red',
    'title': 'H_2_0'
}

config1 = {
    'figsize': figsize,
    'k': k,
    'draw_self_loops': draw_self_loops,
    'node_color': 'orange',
    'node_size': 800,
    'normal_edge_color': 'blue',
    'normal_edge_width': 1.5,
    'edge_style': 'solid',
    'self_loop_color': 'red',
    'title': 'H_2_1'
}

config2 = {
    'figsize': figsize,
    'k': k,
    'draw_self_loops': draw_self_loops,
    'node_color': 'orange',
    'node_size': 800,
    'normal_edge_color': 'blue',
    'normal_edge_width': 1.5,
    'edge_style': 'solid',
    'self_loop_color': 'red',
    'title': 'H_2_2'
}

config3 = {
    'figsize': figsize,
    'k': k,
    'draw_self_loops': draw_self_loops,
    'node_color': 'orange',
    'node_size': 800,
    'normal_edge_color': 'blue',
    'normal_edge_width': 1.5,
    'edge_style': 'solid',
    'self_loop_color': 'red',
    'title': 'H_2_3'
}

In [3]:
# reps = 2
# time = 1

# adjm_cupy= generate_temporal_helix_cupy(reps,time)
# adjm =  generate_temporal_helix(reps,time)
# graph_cupy = nx.from_numpy_array(adjm_cupy.get())
# graph = nx.from_numpy_array(adjm)
# draw_graph(graph)
# draw_graph(graph_cupy)

In [34]:
def estimate_hitting_time_normal(reps, start_vertex, end_vertex, num_simulations=10):
    """
    Estimate the hitting time between two vertices in a list of lollipop graphs.

    Parameters:
    m_values (list): The list of m values, each representing the number of vertices in the complete graph part of a lollipop graph.
    n_values (list): The list of n values, each representing the number of vertices in the path part of a lollipop graph.
    start_vertex (int): The starting vertex for the random walk.
    end_vertex (int): The target vertex for the random walk.
    num_simulations (int): The number of simulations to perform.

    Returns:
    hitting_times (list): A list of estimated average hitting times for each lollipop graph.
    """

    hitting_times = []

    for rep in range(1,reps+1):
        print(f'\nCalculatig hitting time for reps = {rep} ; n = {3+3*rep}')
        total_steps_for_all_simulations = 0

        # Create the lollipop graph
        graph = nx.from_numpy_array(generate_temporal_helix(rep,0))
        current_node = list(graph.nodes)[start_vertex]
        end_node = list(graph.nodes)[end_vertex]
        print(f'Starting node: {current_node} \t Neighbors: {list(nx.neighbors(graph,current_node))}')
        print(f'End node: {end_node} \t Neighbors: {list(nx.neighbors(graph,end_node))}')
        for s in range(num_simulations):
            if s == 0 or s == 1 or s % 5 == 0:
                print(f'----> Sample number:{s}')
                pass
            total_steps_this_simulation = 0
            current_node = list(graph.nodes)[start_vertex]
            # Loop continues until end_vertex is reached
            while current_node != end_node:
                # Choose a neighbor randomly
                graph = nx.from_numpy_array(generate_temporal_helix(rep,total_steps_this_simulation))
                neighbors = list(nx.neighbors(graph, current_node))
                if neighbors:
                    current_node = random.choice(neighbors)
                total_steps_this_simulation += 1
                # print(current_node)
                # print(end_node)
                # print()

            # Accumulate the total steps for this simulation
            total_steps_for_all_simulations += total_steps_this_simulation

        # Average the total steps over the number of simulations
        average_hitting_time = total_steps_for_all_simulations / num_simulations
        hitting_times.append(average_hitting_time)

    return hitting_times

def estimate_hitting_time_cupy(reps, start_vertex, end_vertex, num_simulations=10):
    """
    Estimate the hitting time between two vertices in a list of lollipop graphs.

    Parameters:
    reps (int): The number of repetitions.
    start_vertex (int): The starting vertex for the random walk.
    end_vertex (int): The target vertex for the random walk.
    num_simulations (int): The number of simulations to perform.

    Returns:
    hitting_times (list): A list of estimated average hitting times for each lollipop graph.
    """

    hitting_times = []

    for rep in range(1, reps + 1):
        print(f'\nCalculating hitting time for reps = {rep} ; n = {3 + 3 * rep}')
        total_steps_for_all_simulations = 0

        # Create the lollipop graph
        graph_matrix = generate_temporal_helix_cupy(rep, 0)
        current_node = start_vertex
        end_node = len(graph_matrix) - 1 if end_vertex == -1 else end_vertex
        print(f'Starting node: {current_node} \t Neighbors: {cp.where(graph_matrix[current_node])[0].tolist()}')
        print(f'End node: {end_node} \t Neighbors: {cp.where(graph_matrix[end_node])[0].tolist()}')

        for s in range(num_simulations):
            if s == 0 or s == 1 or s % 5 == 0:
                print(f'----> Sample number:{s}')
            total_steps_this_simulation = 0
            current_node = start_vertex

            # Loop continues until end_vertex is reached
            while current_node != end_node:
                # Update the graph matrix
                graph_matrix = generate_temporal_helix_cupy(rep, total_steps_this_simulation)
                neighbors = cp.where(graph_matrix[current_node])[0]

                # Choose a neighbor randomly
                if neighbors.size > 0:
                    current_node = cp.random.choice(neighbors,size=len(neighbors))[0]
                total_steps_this_simulation += 1

            # Accumulate the total steps for this simulation
            total_steps_for_all_simulations += total_steps_this_simulation

        # Average the total steps over the number of simulations
        average_hitting_time = total_steps_for_all_simulations / num_simulations
        hitting_times.append(average_hitting_time)

    return hitting_times


reps = 5
start_vertex = 0
end_vertex = -1
simul = 1

import time

start_time_normal = time.time()

ht_normal = estimate_hitting_time_normal(reps, start_vertex, end_vertex, simul)

end_time_normal = time.time()
elapsed_time_normal = end_time_normal - start_time_normal

print(f"Time taken for estimate_hitting_time_normal: {elapsed_time_normal} seconds")

start_time_cupy = time.time()

ht_cupy = estimate_hitting_time_cupy(reps, start_vertex, end_vertex, simul)

end_time_cupy = time.time()
elapsed_time_cupy = end_time_cupy - start_time_cupy

print(f"Time taken for estimate_hitting_time_cupy: {elapsed_time_cupy} seconds")


Calculatig hitting time for reps = 1 ; n = 6
Starting node: 0 	 Neighbors: [0, 1]
End node: 5 	 Neighbors: [4, 5]
----> Sample number:0

Calculatig hitting time for reps = 2 ; n = 9
Starting node: 0 	 Neighbors: [0, 1]
End node: 8 	 Neighbors: [7, 8]
----> Sample number:0

Calculatig hitting time for reps = 3 ; n = 12
Starting node: 0 	 Neighbors: [0, 1]
End node: 11 	 Neighbors: [10, 11]
----> Sample number:0

Calculatig hitting time for reps = 4 ; n = 15
Starting node: 0 	 Neighbors: [0, 1]
End node: 14 	 Neighbors: [13, 14]
----> Sample number:0

Calculatig hitting time for reps = 5 ; n = 18
Starting node: 0 	 Neighbors: [0, 1]
End node: 17 	 Neighbors: [16, 17]
----> Sample number:0
Time taken for estimate_hitting_time_normal: 2.556110143661499 seconds

Calculating hitting time for reps = 1 ; n = 6
Starting node: 0 	 Neighbors: [0, 1]
End node: 5 	 Neighbors: [4, 5]
----> Sample number:0

Calculating hitting time for reps = 2 ; n = 9
Starting node: 0 	 Neighbors: [0, 1]
End node: 

KeyboardInterrupt: 